# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

In [44]:
import pandas as pd
url = "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv"
mkcust_df = pd.read_csv(url)
mkcust_df.columns = mkcust_df.columns.str.lower()
mkcust_df.columns = mkcust_df.columns.str.replace(' ','_')
mkcust_df.dtypes

unnamed:_0                         int64
customer                          object
state                             object
customer_lifetime_value          float64
response                          object
coverage                          object
education                         object
effective_to_date                 object
employmentstatus                  object
gender                            object
income                             int64
location_code                     object
marital_status                    object
monthly_premium_auto               int64
months_since_last_claim          float64
months_since_policy_inception      int64
number_of_open_complaints        float64
number_of_policies                 int64
policy_type                       object
policy                            object
renew_offer_type                  object
sales_channel                     object
total_claim_amount               float64
vehicle_class                     object
vehicle_size    

1. Create a new DataFrame that only includes customers who have a total_claim_amount greater than $1,000 and have a response of "Yes" to the last marketing campaign.

In [24]:
display(mkcust_df.shape) # only to check if the initial number of rows
filtered = mkcust_df[(mkcust_df['total_claim_amount'] > 1000) & (mkcust_df['response']=='Yes')] #dataframe filtered by total_claim_amount and response

columns_from_filter = ['customer', 'total_claim_amount', 'response'] #from dataframe filtered extracted the customer, total_claim_amount 
cust_df = filtered[columns_from_filter]#and extracted to the final dataframe
print(cust_df) #print to confirm the accuracy

(10910, 26)

      customer  total_claim_amount response
189    OK31456         1358.400000      Yes
236    YJ16163         1358.400000      Yes
419    GW43195         1027.200000      Yes
442    IP94270         1261.319869      Yes
587    FJ28407         1027.000029      Yes
...        ...                 ...      ...
10351  FN44127         1176.278800      Yes
10373  XZ64172         1324.800000      Yes
10487  IX60941         1176.278800      Yes
10565  QO62792         1008.000000      Yes
10708  CK39096         1027.000029      Yes

[67 rows x 3 columns]


2. Using the original Dataframe, analyze the average total_claim_amount by each policy type and gender for customers who have responded "Yes" to the last marketing campaign. Write your conclusions.

In [30]:
filtered = mkcust_df[(mkcust_df['response']=='Yes')] #filtered the observation with response='yes' from the original dataframe
filtered_group = filtered.groupby(['policy_type','gender'])['total_claim_amount'].mean().reset_index #them presented the average total_claim_amount grouped by the policy_type and gender 
print(filtered_group)

#Conclusions: The most profit policy type is "Personal auto", womem invest more in security for their vehicles and responded more to the last marketing campaign!

<bound method Series.reset_index of policy_type     gender
Corporate Auto  F         433.738499
                M         408.582459
Personal Auto   F         452.965929
                M         457.010178
Special Auto    F         453.280164
                M         429.527942
Name: total_claim_amount, dtype: float64>


3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

In [40]:
filter_customer = mkcust_df.groupby('state')['customer'].count().reset_index()
more_500 = filter_customer[filter_customer['customer']>500]
print(more_500)

        state  customer
0     Arizona      1937
1  California      3552
2      Nevada       993
3      Oregon      2909
4  Washington       888


4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

In [60]:
#"Maximum CLV by education and gender\n")
filter_clv_max = mkcust_df.groupby(['education', 'gender'])['customer_lifetime_value'].max().reset_index()
filter_clv_max.columns = filter_clv_max.columns.str.replace('customer_lifetime_value','maximum clv')

#"Minimum CLV by education and gender\n")
filter_clv_min = mkcust_df.groupby(['education', 'gender'])['customer_lifetime_value'].min().reset_index()
filter_clv_min.columns = filter_clv_min.columns.str.replace('customer_lifetime_value','minimum clv')

#"Median CLV by education and gender\n")
filter_clv_avg = mkcust_df.groupby(['education', 'gender'])['customer_lifetime_value'].median().reset_index()
filter_clv_avg.columns = filter_clv_avg.columns.str.replace('customer_lifetime_value','median clv')

merge_df = pd.merge(filter_clv_max, filter_clv_min, on=['education', 'gender'], how='outer')
merge_df = pd.merge(merge_df, filter_clv_avg, on=['education','gender'], how='outer')
print(merge_df)

#Conclusion:
#Women with Bachelor and Master level have more clv than man in the same level of education, while man dominate the other level with emphasis to HighSchool and below.

              education gender  maximum clv  minimum clv   median clv
0              Bachelor      F  73225.95652  1904.000852  5640.505303
1              Bachelor      M  67907.27050  1898.007675  5548.031892
2               College      F  61850.18803  1898.683686  5623.611187
3               College      M  61134.68307  1918.119700  6005.847375
4                Doctor      F  44856.11397  2395.570000  5332.462694
5                Doctor      M  32677.34284  2267.604038  5577.669457
6  High School or Below      F  55277.44589  2144.921535  6039.553187
7  High School or Below      M  83325.38119  1940.981221  6286.731006
8                Master      F  51016.06704  2417.777032  5729.855012
9                Master      M  50568.25912  2272.307310  5579.099207


## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

In [ ]:
# your code goes here